In [3]:
Amount_of_API_Calls = 0

In [4]:
import pygame
import yfinance as yf , shutil, os , time, glob
import pandas as pd
import numpy as np
import requests
from get_all_tickers import get_tickers as gt
from statistics import mean
from datetime import date
import warnings
warnings.filterwarnings('ignore')

pygame.init()
pygame.mixer.init()
sounda= pygame.mixer.Sound("./../../Storage/VoiceAlerts/Alarms/KZQVUA6-alarm-fire-alarm-buzzer-02.mp3")
soundb= pygame.mixer.Sound("./../../Storage/VoiceAlerts/Alarms/MV27TES-alarm.mp3")
#sounda.play()

In [5]:
df = pd.read_csv("./../../Storage/VoiceAlerts/tables/history.csv")

In [6]:
df

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2023-04-08 00:01:00+00:00,27923.943359,27923.943359,27923.943359,27923.943359,0,0,0
1,2023-04-08 00:02:00+00:00,27923.626953,27923.626953,27923.626953,27923.626953,1760256,0,0
2,2023-04-08 00:03:00+00:00,27921.976562,27921.976562,27921.976562,27921.976562,187392,0,0
3,2023-04-08 00:05:00+00:00,27920.857422,27920.857422,27920.857422,27920.857422,0,0,0
4,2023-04-08 00:06:00+00:00,27920.689453,27920.689453,27920.689453,27920.689453,89088,0,0
...,...,...,...,...,...,...,...,...
8756,2023-04-14 19:10:00+00:00,30270.570312,30270.570312,30270.570312,30270.570312,612352,0,0
8757,2023-04-14 19:11:00+00:00,30261.386719,30261.386719,30261.386719,30261.386719,0,0,0
8758,2023-04-14 19:12:00+00:00,30260.882812,30260.882812,30260.882812,30260.882812,0,0,0
8759,2023-04-14 19:13:00+00:00,30236.382812,30236.382812,30236.382812,30236.382812,0,0,0


In [16]:
def get_RSIf(interval='1m',period='7D'):
    
    #Get the path of each stock in a lsit 
    from turtle import down

    from sympy import Sum


    list_files = (glob.glob("/Users/zoso/Documents/TickerAlerts/PythonAlerts/stock/*.csv"))
    #Yo can use thi line to limit the analysis of a protion of stocks in the folder
    #list_files = list_files[:1]
    # Create the dataframe that we will be adding the final analysis of each stocj to
    Compare_Stocks = pd.DataFrame(columns=["Company", "Days_Observed","Crosses""True_Positive","False_Positive","True_Negative","False_Negative","Sensitivity","Specificity","Accuracy","TPR","FPR"])

    for stock in list_files:
        #DataFrame to fold the historical data of the stock we are interested in.
        Hist_data = pd.read_csv(stock)
        Data = Hist_data.copy()
        Company = ((os.path.basename(stock)).split(".csv")[0]) # name of the company
        # This holds the clsing pricne of a stock
        # while loop to cycle throufhthe stock paths
        #DataFrame to fold the historical data of the stock we are interested in.
        #Company = 'BTC-USD' # name of the company
        # This holds the clsing pricne of a stock
        dates = Data['Datetime'].to_list()
        #volume=Data['Datetime'].to_list()
        prices = Data['Close'].to_list()
        op = Data['Open'].to_list()
        high = Data['High'].to_list()
        low = Data['Low'].to_list()
        c = 0
        
        # add the closing prices to the prices list and make sure we start at greater than 2 dollars to reduce outlier calculations
        '''
        while c< len(Hist_data):
            if Hist_data.iloc[c,4] > float(2.00): # check price fr this day is greater than 2 dollars
                dates.append(str(Hist_data.iloc[c,1]))
                op.append(Hist_data.iloc[c,2])
                prices.append(Hist_data.iloc[c,5])
                high.append(Hist_data.iloc[c,3])
                low.append(Hist_data.iloc[c,4])
                
            c +=1
            print(c)
        '''
        #prices_df = p.DataFrame(prices) # Make a dataframe from price list
        i=0
        upPrices = []
        downPrices = []
        #loop to hold up and down prices movements
        while i < len(prices):
            if i==0:
                upPrices.append(0)
                downPrices.append(0)
            else:
                if (prices[i]-prices[i-1])>0:
                    upPrices.append(prices[i]-prices[i-1])
                    downPrices.append(0)
                else:
                    downPrices.append(prices[i]-prices[i-1])
                    upPrices.append(0)
            i+=1
            #print("i: " +str(i))
        print("finisnshed i")
        x=0
        avg_gain = []
        avg_loss=[]
        #loop to calculate the average gain and loss
        while x < len(upPrices):
            if x<15:
                avg_gain.append(0)
                avg_loss.append(0)    
            else:
                sumGain = 0
                sumLoss = 0
                y=x-14
                while y<=x:
                    sumGain += upPrices[y]
                    sumLoss += downPrices[y]
                    y+=1
                avg_gain.append(sumGain/14)
                avg_loss.append(abs(sumLoss/14))
            x +=1
            #print("x: "+str(x))
        print('finished x')
        p = 0
        RS = []
        RSI = []    
        #Loop to calculate RSI and RS

        while p<len(prices):
            if p<15:
                RS.append(0)
                RSI.append(0)
            else:
                try:
                    RSvalue = (avg_gain[p]/avg_loss[p])
                    RS.append(RSvalue)
                    RSI.append(100-(100/(1+RSvalue)))
                except ZeroDivisionError:
                    RSvalue = np.nan
                    RS.append(RSvalue)
                    RSI.append(100-(100/(1+RSvalue)))
            p+=1
            #print(p)
        #Creates the csv for each stock RSI and prcie movements
        df_dict = {
            'Dates' : dates, 
            'Open': op,
            'High':high,
            'Low':low,
            'Prices' :prices,
            'RSI' : RSI,
            }
        df = pd.DataFrame(df_dict, columns = ['Dates','Open','High','Low','Prices',"RSI"])
        df.to_csv("/Users/zoso/Documents/TickerAlerts/PythonAlerts/RSI/"+Company+"_RSI.csv", index = False)
        abv70 = df[df['RSI']>=70].shape[0] 
        blw30 = df[df['RSI']<=30].shape[0]
        print("The total data above 70 and below 30 RSI is: "+str(abv70)+" and "+str(blw30)+", say, "+str(round(abv70/ df.shape[0]*100,2))+"% and "+str(round(blw30/ df.shape[0]*100,2))+"% of the data. The difference between buy and sell % is: "+str(round(round(abv70/ df.shape[0]*100,2)-round(blw30/ df.shape[0]*100,2),2)))
        print("Last RSI observed : "+str(df.iloc[-1:]["RSI"].to_list()[0]))
        print("gain: "+avg_gain[-7:])
        print("loss: "+avg_loss)[-7:]
        # Code to test the accuracy of the RSI at predicting stock prices
        if df.iloc[-1]['RSI']<30:
            sounda.play()
            print("RSI from "+Company+" below 30")
        elif df.iloc[-1]['RSI']>70:
            soundb.play()
            print("RSI from "+Company+" above 70")
        return df

In [8]:
tickers = ["BTC-USD"] # Establuish tickers to be used 
print("The amount of stock to request is : "+ str(len(tickers)))  # check number of stickers is not bigger than 2000

shutil.rmtree("/Users/zoso/Documents/TickerAlerts/PythonAlerts/stock") # delete the folder 
os.mkdir("/Users/zoso/Documents/TickerAlerts/PythonAlerts/stock") # then recreeate itfro stock

shutil.rmtree("/Users/zoso/Documents/TickerAlerts/PythonAlerts/RSI") # do the same with the rsi
os.mkdir("/Users/zoso/Documents/TickerAlerts/PythonAlerts/RSI") #recreta rsi folder

The amount of stock to request is : 1


In [9]:
stock = tickers[0] # gets current stock ticker
temp = yf.Ticker(str(stock))

In [10]:
temp.history().index

DatetimeIndex(['2023-03-14 00:00:00+00:00', '2023-03-15 00:00:00+00:00',
               '2023-03-16 00:00:00+00:00', '2023-03-17 00:00:00+00:00',
               '2023-03-18 00:00:00+00:00', '2023-03-19 00:00:00+00:00',
               '2023-03-20 00:00:00+00:00', '2023-03-21 00:00:00+00:00',
               '2023-03-22 00:00:00+00:00', '2023-03-23 00:00:00+00:00',
               '2023-03-24 00:00:00+00:00', '2023-03-25 00:00:00+00:00',
               '2023-03-26 00:00:00+00:00', '2023-03-27 00:00:00+00:00',
               '2023-03-28 00:00:00+00:00', '2023-03-29 00:00:00+00:00',
               '2023-03-30 00:00:00+00:00', '2023-03-31 00:00:00+00:00',
               '2023-04-01 00:00:00+00:00', '2023-04-02 00:00:00+00:00',
               '2023-04-03 00:00:00+00:00', '2023-04-04 00:00:00+00:00',
               '2023-04-05 00:00:00+00:00', '2023-04-06 00:00:00+00:00',
               '2023-04-07 00:00:00+00:00', '2023-04-08 00:00:00+00:00',
               '2023-04-09 00:00:00+00:00', '2023-0

In [11]:
Stock_Failure = 0
Stocks_Not_Imported = 0

# used to iteretae thorugh our lsit of tickers
i=0
while (i < len(tickers)) and (Amount_of_API_Calls < 1800):
    try:
        print("Iteration=" +str(i))
        stock = tickers[i] # gets current stock ticker
        temp = yf.Ticker(str(stock))

        Hist_data = temp.history(period="max")
        Hist_data.to_csv("/Users/zoso/Documents/TickerAlerts/PythonAlerts/stock/data_"+tickers[i]+"_"+str(date.today())+"_1D_max.csv") # Save the historical data as a cdv data for further procsessing
        
        Hist_data = temp.history(interval="1m",period="7d")
        Hist_data.to_csv("/Users/zoso/Documents/TickerAlerts/PythonAlerts/stock/data_"+tickers[i]+"_"+str(date.today())+"_1m_7D.csv") # Save the historical data as a cdv data for further procsessing
        Hist_data = temp.history(interval="2m",period="60d")
        Hist_data.to_csv("/Users/zoso/Documents/TickerAlerts/PythonAlerts/stock/data_"+tickers[i]+"_"+str(date.today())+"_2m_60D.csv") # Save the historical data as a cdv data for further procsessing
        Hist_data = temp.history(interval="5m",period="60d")
        Hist_data.to_csv("/Users/zoso/Documents/TickerAlerts/PythonAlerts/stock/data_"+tickers[i]+"_"+str(date.today())+"_5m_60D.csv") # Save the historical data as a cdv data for further procsessing
        Hist_data = temp.history(interval="15m",period="60d")
        Hist_data.to_csv("/Users/zoso/Documents/TickerAlerts/PythonAlerts/stock/data_"+tickers[i]+"_"+str(date.today())+"_15m_60D.csv") # Save the historical data as a cdv data for further procsessing
        Hist_data = temp.history(interval="30m",period="60d")
        Hist_data.to_csv("/Users/zoso/Documents/TickerAlerts/PythonAlerts/stock/data_"+tickers[i]+"_"+str(date.today())+"_30m_60D.csv") # Save the historical data as a cdv data for further procsessing
        Hist_data = temp.history(interval="1h",period="730d")
        Hist_data.to_csv("/Users/zoso/Documents/TickerAlerts/PythonAlerts/stock/data_"+tickers[i]+"_"+str(date.today())+"_60m_730D.csv") # Save the historical data as a cdv data for further procsessing
        Hist_data = temp.history(interval="90m",period="60d")
        Hist_data.to_csv("/Users/zoso/Documents/TickerAlerts/PythonAlerts/stock/data_"+tickers[i]+"_"+str(date.today())+"_90m_60D.csv") # Save the historical data as a cdv data for further procsessing
        Hist_data = temp.history(interval="5d",period="max")
        Hist_data.to_csv("/Users/zoso/Documents/TickerAlerts/PythonAlerts/stock/data_"+tickers[i]+"_"+str(date.today())+"_5d_max.csv") # Save the historical data as a cdv data for further procsessing
        Hist_data = temp.history(interval="1wk",period="max")
        Hist_data.to_csv("/Users/zoso/Documents/TickerAlerts/PythonAlerts/stock/data_"+tickers[i]+"_"+str(date.today())+"_1wk_max.csv") # Save the historical data as a cdv data for further procsessing
        Hist_data = temp.history(interval="1mo",period="max")
        Hist_data.to_csv("/Users/zoso/Documents/TickerAlerts/PythonAlerts/stock/data_"+tickers[i]+"_"+str(date.today())+"_1mo_max.csv") # Save the historical data as a cdv data for further procsessing
        Hist_data = temp.history(interval="3mo",period="max")
        Hist_data.to_csv("/Users/zoso/Documents/TickerAlerts/PythonAlerts/stock/data_"+tickers[i]+"_"+str(date.today())+"_3mo_max.csv") # Save the historical data as a cdv data for further procsessing
        
        time.sleep(2) # Make a pause of two seconds so do  not make a casue with yfinance operations
        Amount_of_API_Calls +=1 
    
        Stock_Failure = 0
        i +=1 #Iteration to the next ticker
    except ValueError:
        print("Yahoo Finance Bckend Error, Attempting to Fix")
        if Stock_Failure > 5:
            i+=1
            Stocks_Not_Imported +=1
        Amount_of_API_Calls +=1
        Stock_Failure +=1
    #Handle SSL Error
    except requests.exceptions.SSLError as e:
        print("Yahoo Finance Backend Error, Attempting to Fix SSL")# An error ocurred on Yahhoo Finance backen,. We will ATTEMPT TO RETRIEVE THE DATA AGAIN
        if Stock_Failure > 5:
            i+=1
            Stocks_notImported +=1
        Amount_of_API_Calls +=1
        Stock_Failure +=1
    print("The amount of stocks we succesfully imported: "+ str(i-Stocks_Not_Imported))
    





Iteration=0
- BTC-USD: No data found for this date range, symbol may be delisted
- BTC-USD: No data found for this date range, symbol may be delisted
- BTC-USD: No data found for this date range, symbol may be delisted
- BTC-USD: No data found for this date range, symbol may be delisted


KeyboardInterrupt: 

In [46]:
#Get the path of each stock in a lsit 
from turtle import down

from sympy import Sum

def get_RSI_alarm(df,Company):
    
    #list_files = (glob.glob("/Users/zoso/Documents/TickerAlerts/PythonAlerts/stock/*.csv"))
    #Yo can use thi line to limit the analysis of a protion of stocks in the folder
    #list_files = list_files[:1]
    # Create the dataframe that we will be adding the final analysis of each stocj to
    #Compare_Stocks = pd.DataFrame(columns=["Company", "Days_Observed","Crosses""True_Positive","False_Positive","True_Negative","False_Negative","Sensitivity","Specificity","Accuracy","TPR","FPR"])
    list_files=['ganbare']
    # while loop to cycle throufhthe stock paths
    for stock in list_files:
        #DataFrame to fold the historical data of the stock we are interested in.
        Hist_data = df
        Company = Company # name of the company
        # This holds the clsing pricne of a stock
        dates = []
        volume = []
        prices = []
        c = 0
        # add the closing prices to the prices list and make sure we start at greater than 2 dollars to reduce outlier calculations
        while c< len(Hist_data):
            if Hist_data.iloc[c,4] > float(2.00): # check price fr this day is greater than 2 dollars
                dates.append(Hist_data.iloc[c,0])
                volume.append(Hist_data.iloc[c,5])
                prices.append(Hist_data.iloc[c,4])
            c +=1
        #prices_df = p.DataFrame(prices) # Make a dataframe from price list
        i=0
        upPrices = []
        downPrices = []
        #loop to hold up and down prices movements
        while i < len(prices):
            if i==0:
                upPrices.append(0)
                downPrices.append(0)
            else:
                if (prices[i]-prices[i-1])>0:
                    upPrices.append(prices[i]-prices[i-1])
                    downPrices.append(0)
                else:
                    downPrices.append(prices[i]-prices[i-1])
                    upPrices.append(0)
            i+=1
            #print("i: "+str(i))
        x=0
        avg_gain = []
        avg_loss=[]
        #loop to calculate the average gain and loss
        while x < len(upPrices):
            if x<15:
                avg_gain.append(0)
                avg_loss.append(0)    
            else:
                sumGain = 0
                sumLoss = 0
                y=x-14
                while y<=x:
                    sumGain += upPrices[y] # sum of last 14 periods for down prices
                    sumLoss += downPrices[y] # sum of last 14 periods for up prices
                    y+=1 
                avg_gain.append(sumGain/14) # take average of gain
                avg_loss.append(abs(sumLoss/14)) # take average of loss
            x +=1
        p = 0
        RS = []
        RSI = []    
        #Loop to calculate RSI and RS
        while p<len(prices):
            if p<15:
                RS.append(0)
                RSI.append(0)
            else:
                RSvalue = (avg_gain[p]/avg_loss[p])
                RS.append(RSvalue)
                RSI.append(100-(100/(1+RSvalue)))
            p+=1
        #Creates the csv for each stock RSI and prcie movements
        df_dict = {
            'Dates': dates,
            'Prices' :prices,
            'upPrices' : upPrices,
            'downPrices' : downPrices,
            'AvgGain' : avg_gain,
            'AvgLoss' : avg_loss,
            'RS':RS,
            'RSI' : RSI
            }
        df = pd.DataFrame(df_dict, columns = ['Dates','Prices', 'upPrices', 'downPrices', 'AvgGain','AvgLoss', 'RS', "RSI"])
        df.to_csv(f"./{Company}_RSI.csv", index = False)
        print(dates[-1])
        print(Company)
        abv70 = df[df['RSI']>=70].shape[0] 
        blw30 = df[df['RSI']<=30].shape[0]
        print("The total data above 70 and below 30 RSI is: "+str(abv70)+" and "+str(blw30)+", say, "+str(round(abv70/ df.shape[0]*100,2))+"% and "+str(round(blw30/ df.shape[0]*100,2))+"% of the data. The difference between buy and sell % is: "+str(round(round(abv70/ df.shape[0]*100,2)-round(blw30/ df.shape[0]*100,2),2))+" with last RSI iberved being: "+str(df.iloc[-1:]["RSI"].to_list()[0]))
        if df.iloc[-1]['RSI']<35:
            sounda.play()
            print("RSI from "+Company+" below 30")
        elif df.iloc[-1]['RSI']>70:
            soundb.play()
            print("RSI from "+Company+" above 70")
    return df


In [47]:
df[['RS','RSI']][-20:]
if df.iloc[-1]['RSI']<30:
    sounda.play()
elif df.iloc[-1]['RSI']>70:
    soundb.play()


KeyError: "None of [Index(['RS', 'RSI'], dtype='object')] are in the [columns]"

In [48]:
pruv = get_RSI_alarm(df,'BTC-USD')


2023-04-14 19:14:00+00:00
BTC-USD
The total data above 70 and below 30 RSI is: 1316 and 1089, say, 15.02% and 12.43% of the data. The difference between buy and sell % is: 2.59 with last RSI iberved being: 33.319997418855266
RSI from BTC-USD below 30


In [56]:
import pandas as pd
from pyrsistent import v
import yfinance as yf
import numpy as np
import pygame
import warnings
warnings.filterwarnings('ignore')
import time

from turtle import down
from sympy import Sum
import glob

def get_RSI(ticker,interval='1D',period='max',rsi_periods=14):
    
    
    #Yo can use thi line to limit the analysis of a protion of stocks in the folder
    #list_files = list_files[:1]
    # Create the dataframe that we will be adding the final analysis of each stocj to

    # while loop to cycle throufhthe stock paths
    #DataFrame to fold the historical data of the stock we are interested in.
    Company = ticker# name of the company
    Datah = yf.Ticker(ticker)
    Data = Datah.history(interval=interval,period=period)
    # This holds the clsing pricne of a stock
    dates = Data.index.to_list()
    vol=Data['Volume'].to_list()
    prices = Data['Close'].to_list()
    op = Data['Open'].to_list()
    high = Data['High'].to_list()
    low = Data['Low'].to_list()
    c = 0
    #prices_df = p.DataFrame(prices) # Make a dataframe from price list



    #Compute avg gain and loss
    i=0
    upPrices = []
    downPrices = []
    #loop to hold up and down prices movements
    while i < len(prices):
        if i==0:
            upPrices.append(0)
            downPrices.append(0)
        else:
            if (prices[i]-prices[i-1])>0:
                upPrices.append(prices[i]-prices[i-1])
                downPrices.append(0)
            else:
                downPrices.append(prices[i]-prices[i-1])
                upPrices.append(0)
        i+=1
        #print("i: " +str(i))
    #print("finisnshed i")
    x=0
    avg_gain = []
    avg_loss=[]
    #loop to calculate the average gain and loss
    while x < len(upPrices):
        if x< (rsi_periods+1):
            avg_gain.append(0)
            avg_loss.append(0)    
        else:
            sumGain = 0
            sumLoss = 0
            y=x-rsi_periods
            while y<=x:
                sumGain += upPrices[y]
                sumLoss += downPrices[y]
                y+=1
            avg_gain.append(sumGain/rsi_periods)
            avg_loss.append(abs(sumLoss/rsi_periods))
        x +=1
        #print("x: "+str(x))
    #print('finished x')

#Compute RSI:

    p = 0
    RS = []
    RSI = []    
    #Loop to calculate RSI and RS

    while p<len(prices):
        if p<15:
            RS.append(0)
            RSI.append(0)
        else:
            try:
                RSvalue = (avg_gain[p]/avg_loss[p])
                RS.append(RSvalue)
                RSI.append(100-(100/(1+RSvalue)))
            except ZeroDivisionError:
                RSvalue = np.nan
                RS.append(RSvalue)
                RSI.append(100-(100/(1+RSvalue)))
        p+=1
        #print(p)
    #Creates the csv for each stock RSI and prcie movements
    df_dict = {
        'Dates' : dates, 
        'Open': op,
        'High':high,
        'Low':low,
        'Prices' :prices,
        'RSI' : RSI,
        'RS':RS,
        'upPrices': upPrices,
        'downPrices': downPrices,
        'avg_gain': avg_gain,
        'avg_loss':avg_loss,
        'Volume':vol
        }
    df = pd.DataFrame(df_dict, columns = ['Dates','Open','High','Low','Prices',"RSI",'RS','upPrices','downPrices','avg_gain','avg_loss','Volume'])
    df.to_csv("/Users/zoso/Documents/TickerAlerts/PythonAlerts/RSI/"+Company+"_RSI.csv", index = False)
    df['RSI'] = round(df['RSI'])
    abv70 = df[df['RSI']>=70].shape[0] 
    blw30 = df[df['RSI']<=30].shape[0]
    abvpr = round(abv70/ df.shape[0]*100,2)
    blwpr = round(blw30/ df.shape[0]*100,2)
    diff = abvpr - blwpr
    DiffRSI = df['RSI'].iloc[-1] - df['RSI'].iloc[-2]
    DiffPrice = df.iloc[-1]['Prices'] - df.iloc[-2]['Prices']
    DiffPricePercent = round(DiffPrice/df.iloc[-1]['Prices']*100,2)

    print(df['Dates'].iloc[-1])
    print("Price: "+str(df['Prices'].iloc[-1]))
    print("The total data above 70 and below 30 RSI is: "+str(abv70)+" and "+str(blw30)+", say, "+str(abvpr)+"% and "+str(blwpr)+"% of the data.")
    print("The difference between buy and sell % is: "+str(round(diff,2)))
    print("Values observed: "+str(df.shape[0]))
    print("RSI observed : "+str(df.iloc[-1]["RSI"]))
    print("Volume observed: "+str(df['Volume'].iloc[-1]) )
    print("| (upPrices,downPrices): "+str((df['upPrices'].iloc[-1],df['downPrices'].iloc[-1])))
    print("| (avgGain,avgLoss): " +str((df['avg_gain'].iloc[-1],df['avg_loss'].iloc[-1])))
    print("RSI Difference is: "+str(DiffRSI))
    print("Price Difference is: "+str(DiffPrice))
    print("An increment of  "+str(DiffPricePercent)+"%")
    
    return df 


def get_Price_Alerts(ticker,minutes=1,interval=['1m'],period=['7D'],abvl=100,blwl=0,time_sleep=60):
    i=0
    while i < minutes:
        for tick in range(len(ticker)):
            for inter in range(len(interval)):
                print(ticker[tick],interval[inter])
                Datah = yf.Ticker(ticker[tick])
                df = Datah.history(interval=interval[inter],period=period[inter])

                #Increments
                if (df.iloc[-1]['Close'] > (1.1*(df.iloc[-2]['Close']))) | ((df.iloc[-1]['Close'] > (1.1*(df.iloc[-3]['Close'])))) | ((df.iloc[-1]['Close'] > (1.1*(df.iloc[-4]['Close'])))) | ((df.iloc[-1]['Close'] > (1.02*(df.iloc[-5]['Close'])))) | ((df.iloc[-1]['Close'] > (1.1*(df.iloc[-6]['Close'])))):
                    if interval[inter] == '1m':
                        a120.play()
                    elif interval[inter] =='15m':
                        a121.play()
                    elif interval[inter] =='1H':
                        a122.play()
                    print("check")

                elif (df.iloc[-1]['Close'] > (1.05*(df.iloc[-2]['Close']))) | ((df.iloc[-1]['Close'] > (1.05*(df.iloc[-3]['Close'])))) | ((df.iloc[-1]['Close'] > (1.05*(df.iloc[-4]['Close'])))):
                    if interval[inter] == '1m':
                        a112.play()
                    elif interval[inter] =='5m':
                        a113.play()
                    elif interval[inter] =='10m':
                        a114.play()
                    elif interval[inter] =='15m':
                        a115.play()
                    elif interval[inter] =='30m':
                        a116.play()
                    elif interval[inter] =='1H':
                        a117.play()
                    elif interval[inter] =='1D':
                        a118.play()
                    elif interval[inter] =='1wk':
                        a119.play()
                    print("check")
                   
                elif (df.iloc[-1]['Close'] > (1.02*(df.iloc[-2]['Close']))) | ((df.iloc[-1]['Close'] > (1.02*(df.iloc[-3]['Close'])))):
                    if interval[inter] == '1m':
                        a88.play()
                    elif interval[inter] =='5m':
                        a89.play()
                    elif interval[inter] =='10m':
                        a90.play()
                    elif interval[inter] =='15m':
                        a91.play()
                    elif interval[inter] =='30m':
                        a92.play()
                    elif interval[inter] =='1H':
                        a93.play()
                    elif interval[inter] =='1D':
                        a94.play()
                    elif interval[inter] =='1wk':
                        a95.play()
                    print("check")

                elif (df.iloc[-1]['Close'] > df.iloc[-2]['Close']):
                    if interval[inter] == '1m':
                        a72.play()
                    elif interval[inter] =='5m':
                        a73.play()
                    elif interval[inter] =='10m':
                        a74.play()
                    elif interval[inter] =='15m':
                        a75.play()
                    elif interval[inter] =='30m':
                        a76.play()
                    elif interval[inter] =='1H':
                        a77.play()
                    elif interval[inter] =='1D':
                        a78.play()
                    elif interval[inter] =='1wk':
                        a79.play()
                
            
                #Decrements
                
                elif (df.iloc[-1]['Close'] < (0.9*(df.iloc[-2]['Close']))) | ((df.iloc[-1]['Close'] < (0.9*(df.iloc[-3]['Close'])))) | ((df.iloc[-1]['Close'] < (0.9*(df.iloc[-4]['Close'])))) | ((df.iloc[-1]['Close'] < (0.9*(df.iloc[-5]['Close'])))) | ((df.iloc[-1]['Close'] < (0.9*(df.iloc[-6]['Close'])))):
                    if interval[inter] == '1m':
                        a123.play()
                    elif interval[inter] =='15m':
                        a124.play()
                    elif interval[inter] =='1H':
                        a125.play()
                    print("check")

                elif (df.iloc[-1]['Close'] < (0.95*(df.iloc[-2]['Close']))) | ((df.iloc[-1]['Close'] < (0.95*(df.iloc[-3]['Close'])))) | ((df.iloc[-1]['Close'] < (0.95*(df.iloc[-4]['Close'])))):
                    if interval[inter] == '1m':
                        a104.play()
                    elif interval[inter] =='5m':
                        a105.play()
                    elif interval[inter] =='10m':
                        a106.play()
                    elif interval[inter] =='15m':
                        a107.play()
                    elif interval[inter] =='30m':
                        a108.play()
                    elif interval[inter] =='1H':
                        a109.play()
                    elif interval[inter] =='1D':
                        a110.play()
                    elif interval[inter] =='1wk':
                        a111.play()
                    print("check")
                   
                elif (df.iloc[-1]['Close'] < (0.98*(df.iloc[-2]['Close']))) | ((df.iloc[-1]['Close'] < (0.98*(df.iloc[-3]['Close'])))):
                    pass
                    '''
                    if interval[inter] == '1m':
                        a96.play()
                    elif interval[inter] =='5m':
                        a97.play()
                    elif interval[inter] =='10m':
                        a98.play()
                    elif interval[inter] =='15m':
                        a99.play()
                    elif interval[inter] =='30m':
                        a100.play()
                    elif interval[inter] =='1H':
                        a101.play()
                    elif interval[inter] =='1D':
                        a102.play()
                    elif interval[inter] =='1wk':
                        a103.play()
                    print("check")
                    '''

                elif (df.iloc[-1]['Close'] < df.iloc[-2]['Close']):
                    if interval[inter] == '1m':
                        a80.play()
                    elif interval[inter] =='5m':
                        a81.play()
                    elif interval[inter] =='10m':
                        a82.play()
                    elif interval[inter] =='15m':
                        a83.play()
                    elif interval[inter] =='30m':
                        a84.play()
                    elif interval[inter] =='1H':
                        a85.play()
                    elif interval[inter] =='1D':
                        a86.play()
                    elif interval[inter] =='1wk':
                        a87.play()
                    print("check price down")
                elif ((df.iloc[-1]['Close'] < 0.8*df.iloc[-2]['Close'])):
                    a126.play()
                
                
                time.sleep(2)
                print("-"*70)
            print("*"*70)
        i+=1
        time.sleep(time_sleep) # 60 seconds for 1 minute wait
        return df

def get_RSI_Alerts(ticker,vd,minutes=1,interval=['1m'],period=['7D'],abvl=100,blwl=0,time_sleep=60):
    i=0
    while i < minutes:
        for tick in range(len(ticker)):
            for inter in range(len(interval)):
                print(ticker[tick],interval[inter])
                df = get_RSI(ticker[tick],interval=interval[inter],period=period[inter])
                #get_Price_Alerts(ticker=ticker,minutes=1,interval=interval,period=period,time_sleep=0)
                time.sleep(1)
                
                if df['RSI'].iloc[-1] > 95:
                    if interval[inter] == '1m':
                        sound95.play()
                    elif interval[inter] =='5m':
                        a18.play()
                    elif interval[inter] =='10m':
                        a19.play()
                    elif interval[inter] =='15m':
                        a20.play()
                    elif interval[inter] =='30m':
                        a21.play()
                    elif interval[inter] =='1H':
                        a22.play()
                    elif interval[inter] =='1D':
                        a23.play()
                    elif interval[inter] =='1wk':
                        a24.play()

                elif df['RSI'].iloc[-1] >90:
                    if interval[inter] == '1m':
                        sound90.play()
                    elif interval[inter] =='5m':
                        a26.play()
                    elif interval[inter] =='10m':
                        a27.play()
                    elif interval[inter] =='15m':
                        a28.play()
                    elif interval[inter] =='30m':
                        a29.play()
                    elif interval[inter] =='1H':
                        a30.play()
                    elif interval[inter] =='1D':
                        a31.play()
                    elif interval[inter] =='1wk':
                        a32.play()

                elif df['RSI'].iloc[-1] >80:
                    if interval[inter] == '1m':
                        sound80.play()
                    elif interval[inter] =='5m':
                        a34.play()
                    elif interval[inter] =='10m':
                        a35.play()
                    elif interval[inter] =='15m':
                        a36.play()
                    elif interval[inter] =='30m':
                        a37.play()
                    elif interval[inter] =='1H':
                        a38.play()
                    elif interval[inter] =='1D':
                        a39.play()
                    elif interval[inter] =='1wk':
                        a40.play()

                elif df['RSI'].iloc[-1] >=70:
                    if interval[inter] == '1m':
                        sound70.play()
                    elif interval[inter] =='5m':
                        a10.play()
                    elif interval[inter] =='10m':
                        a11.play()
                    elif interval[inter] =='15m':
                        a12.play()
                    elif interval[inter] =='30m':
                        a13.play()
                    elif interval[inter] =='1H':
                        a14.play()
                    elif interval[inter] =='1D':
                        a15.play()
                    elif interval[inter] =='1wk':
                        a16.play()
                    
                elif df['RSI'].iloc[-1] < 10:
                    if interval[inter] == '1m':
                        sound10.play()
                    elif interval[inter] =='5m':
                        a42.play()
                    elif interval[inter] =='10m':
                        a43.play()
                    elif interval[inter] =='15m':
                        a44.play()
                    elif interval[inter] =='30m':
                        a45.play()
                    elif interval[inter] =='1H':
                        a46.play()
                    elif interval[inter] =='1D':
                        a47.play()
                    elif interval[inter] =='1wk':
                        a48.play()

                elif df['RSI'].iloc[-1] < 20:
                    if interval[inter] == '1m':
                        sound20.play()
                    elif interval[inter] =='5m':
                        a50.play()
                    elif interval[inter] =='10m':
                        a51.play()
                    elif interval[inter] =='15m':
                        a52.play()
                    elif interval[inter] =='30m':
                        a53.play()
                    elif interval[inter] =='1H':
                        a54.play()
                    elif interval[inter] =='1D':
                        a55.play()
                    elif interval[inter] =='1wk':
                        a56.play()

                elif df['RSI'].iloc[-1] < 25:
                    sound25.play()
                #RSI below 30
                elif df['RSI'].iloc[-1] <= 30:
                    if interval[inter] == '1m':
                        sound30.play()
                    elif interval[inter] =='5m':
                        a2.play()
                    elif interval[inter] =='10m':
                        a3.play()
                    elif interval[inter] =='15m':
                        a4.play()
                    elif interval[inter] =='30m':
                        a5.play()
                    elif interval[inter] =='1H':
                        a6.play()
                    elif interval[inter] =='1D':
                        a7.play()
                    elif interval[inter] =='1wk':
                        a8.play()
                # RSI touches 50
                elif df['RSI'].iloc[-1] in range(49,51):
                    if interval[inter] == '1m':
                        a64.play()
                    elif interval[inter] =='5m':
                        a65.play()
                    elif interval[inter] =='10m':
                        a66.play()
                    elif interval[inter] =='15m':
                        a67.play()
                    elif interval[inter] =='30m':
                        a68.play()
                    elif interval[inter] =='1H':
                        a69.play()
                    elif interval[inter] =='1D':
                        a70.play()
                    elif interval[inter] =='1wk':
                        a71.play()
                
                time.sleep(1)
                print("-"*70)
            print("*"*70)
        i+=1
        time.sleep(time_sleep) # 60 seconds for 1 minute wait
        return df

def get_price_prob(df,roun=2):
    df['Prices'] = round(df['Prices'],roun)
    li = (df.groupby(by=df['Prices']).count().sort_values(by='Open',ascending=False).index)
    dict = {}
    for i in range(len(li)):
        dict[df['Prices'][i]]=np.nan
    print("Price Probabilities are as follow: ")
    for i in li:
        val = (df['Prices'] == i).sum()/df.shape[0] *100
        dict[i]=round(val,2)
        print(str(i)+": "+str(round(val,2))+"%")
    probdf = pd.DataFrame(data=dict.values(),columns=['Price Probability'],index=dict.keys())
    print("-"*70)
    print("*"*70)
    return dict , probdf

def get_RSI_prob(df):
    li = list(df.groupby(by='RSI').count().sort_values(by='Open',ascending=False).index)
    dict = {}
    for i in range(len(li)):
        dict[df['RSI'][i]]=np.nan
    print("RSI Probabilities are as follow: ")
    for i in li:
        val = (df['RSI'] == i).sum()/df.shape[0] *100
        dict[df['RSI'][i]]=round(val,2)
        print(str(i)+": "+str(round(val,2))+"%")
    print("-"*70)
    print("*"*70)
    
    probdf = pd.DataFrame(data=dict.values(),columns=['Price Probability'],index=dict.keys())
    return dict , probdf

def get_voiceLinks(path="/Users/zoso/documents/Courses\ and\ Certifications\/Carreer\ Path\/PythonAlerts/vAlerts/*.mp3"):
    list_files = (glob.glob(path))
    #list_file = list_files.sort()
    li = []
    dict ={}
    for i in range(len(list_files)):
        if len(list_files[i]) == 89:
            val = list_files[i][-6:-4]
        elif len(list_files[i]) == 90:
            val = (list_files[i])[-7:-4]
        elif len(list_files[i]) == 91:
            val = (list_files[i])[-8:-4]
        else:
            print("Error creating list_files for voices")
        li.append(val)
        
        for voice in range(len(li)):
            dict[li[voice]] = list_files[voice]
    return dict
    

#Init alarms
pygame.init()
pygame.mixer.init()

dv = get_voiceLinks()
'./../../Storage/VoiceAlerts/vAlerts/'
a1 = pygame.mixer.Sound(dv["/Users/zoso/Documents/Courses_and_Certifications/Carreer Path/BTC_DATA/Storage/VoiceAlerts/vAlerts/a1"])
a2 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a2"])
a3 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a3"])
a4 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a4"])
a5 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a5"])
a6 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a6"])
a7 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a7"])
a8 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a8"])
a9 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a9"])
a10 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a10"])
a11 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a11"])
a12 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a12"])
a13 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a13"])
a14 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a14"])
a15 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a15"])
a16 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a16"])
a17 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a17"])
a18 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a18"])
a19 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a19"])
a20 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a20"])
a21 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a21"])
a22 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a22"])
a23 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a23"])
a24 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a24"])
a25 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a25"])
a26 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a26"])
a27 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a27"])
a28 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a28"])
a29 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a29"])
a30 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a30"])
a31 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a31"])
a32 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a32"])
a33 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a33"])
a34 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a34"])
a35 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a35"])
a36 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a36"])
a37 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a37"])
a38 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a38"])
a39 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a39"])
a40 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a40"])
a41 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a41"])
a42 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a42"])
a43 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a43"])
a44 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a44"])
a45 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a45"])
a46 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a46"])
a47 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a47"])
a48 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a48"])
a49 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a49"])
a50 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a50"])
a51 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a51"])
a52 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a52"])
a53 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a53"])
a54 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a54"])
a55 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a55"])
a56 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a56"])
a57 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a57"])
a58 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a58"])
a59 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a59"])
a60 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a60"])
a61 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a61"])
a62 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a62"])
a63 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a63"])
a64 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a64"])
a65 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a65"])
a66 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a66"])
a67 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a67"])
a68 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a68"])
a69 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a69"])
a70 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a70"])
a71 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a71"])
a72 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a72"])
a73 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a73"])
a74 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a74"])
a75 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a75"])
a76 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a76"])
a77 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a77"])
a78 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a78"])
a79 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a79"])
a80 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a80"])
a81 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a81"])
a82 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a82"])
a83 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a83"])
a84 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a84"])
a85 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a85"])
a86 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a86"])
a87 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a87"])
a88 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a88"])
a89 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a89"])
a90 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a90"])
a91 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a91"])
a92 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a92"])
a93 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a93"])
a94 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a94"])
a95 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a95"])
a96 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a96"])
a97 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a97"])
a98= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a98"])
a99 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a99"])
a100 = pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a100"])
a101= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a101"])
a102= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a102"])
a103= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a103"])
a104= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a104"])
a105= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a105"])
a106= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a106"])
a107= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a107"])
a108= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a108"])
a109= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a109"])
a110= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a110"])
a111= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a111"])
a112= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a112"])
a113= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a113"])
a114= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a114"])
a115= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a115"])
a116= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a116"])
a117= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a117"])
a118= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a118"])
a119= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a119"])
a120= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a120"])
a121= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a121"])
a122= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a122"])
a123= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a123"])
a124= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a124"])
a125= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a125"])
a126= pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a126"])


sound30= pygame.mixer.Sound("./../../Storage/VoiceAlerts/voiceAlert/RSI_blw30.mp3")
sound70= pygame.mixer.Sound("./../../Storage/VoiceAlerts/voiceAlert/RSI_abv70.mp3")
sound50=pygame.mixer.Sound("./../../Storage/VoiceAlerts/voiceAlert/RSI_50.mp3")
sound80=pygame.mixer.Sound("./../../Storage/VoiceAlerts/voiceAlert/RSI_abv80_1m.mp3")
sound90=pygame.mixer.Sound("./../../Storage/VoiceAlerts/voiceAlert/RSI_abv90_1m.mp3")
sound95=pygame.mixer.Sound("./../../Storage/VoiceAlerts/voiceAlert/RSI_abv95_1m.mp3")
sound25=pygame.mixer.Sound("./../../Storage/VoiceAlerts/voiceAlert/RSI_blw25_1m.mp3")
sound20=pygame.mixer.Sound("./../../Storage/VoiceAlerts/voiceAlert/RSI_blw20_1m.mp3")
sound10=pygame.mixer.Sound("./../../Storage/VoiceAlerts/voiceAlert/RSI_blw10_1m.mp3")
#sounda.play()

tickers = ['BTC-USD']
minutes = 100
interval = ['1m','5m','15m','30m','1H','1D']
period = ['7D','60d','60d','60d','720d','720d']
mins=60


i=0
while i < mins:
    for tick in range(len(tickers)):
        #print(tick)
        for inter in range(len(interval)):
            # function get_RSI already calls het_Price_Alerts inside
            df = get_RSI_Alerts([tickers[tick]],dv,minutes,[interval[inter]],[period[inter]],time_sleep=0)#['1m','5m','15m','30m','1H','1D','1wk'],['7D','60d','60d','60d','720d','720d','1500d'])
        #get_RSI_prob(df)
        #get_price_prob(df)
    i+=1
    time.sleep(45)


KeyError: '/Users/zoso/Documents/Courses_and_Certifications/Carreer Path/BTC_DATA/Storage/VoiceAlerts/vAlerts/a1'

In [54]:
/Users/zoso/Documents/Courses_and_Certifications/Carreer Path/BTC_DATA/Storage/VoiceAlerts/vAlerts

NameError: name 'Users' is not defined

In [60]:
pygame.mixer.Sound(dv["./../../Storage/VoiceAlerts/vAlerts/a1.mp3"])


KeyError: './../../Storage/VoiceAlerts/vAlerts/a1.mp3'